In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
import netCDF4
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from torch.utils.data import DataLoader, random_split
from iwc2tb.GMI.gmiData import gmiData
import os

In [2]:
from typhon.retrieval.qrnn import set_backend, QRNN
set_backend("pytorch")

quantiles         = np.array([0.002, 0.03, 0.16, 0.5, 0.84, 0.97, 0.998])
#quantiles         = np.arange(0, 1, 0.1)

batchSize         = 256

depth             = 4
width             = 256
convergence_epoch = 7
maximum_epoch     = 60

inputs            = 6

Using TensorFlow backend.


In [3]:
inChannels = np.array(['166.5V', '166.5H', '183+-3', '183+-7'], dtype=object)


In [ ]:
def train(depth, width, batchSize, convergence_epoch, maximum_epoch, training_data, validation_data):
        qrnn = QRNN(inputs, quantiles, (depth, width , "relu"))
        for lr in [  0.01, 0.001, 0.0001]:
            print ("NEW LEARNING RATE")
            results = qrnn.train(
                training_data,
                validation_data,
                batch_size=batchSize,
                momentum = 0,
                sigma_noise=None,
                initial_learning_rate= lr ,
                maximum_epochs=maximum_epoch,
                convergence_epochs= convergence_epoch,    
                gpu=True)

        return results, qrnn

### read data

In [4]:

data = gmiData(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/TB_GMI_train.nc"), inChannels,
               batch_size = batchSize) 

In [ ]:
 

n = len(data)
n_train = int(0.9 * n)
n_val = n - n_train

training_data, validation_data = random_split(data, [n_train, n_val])
results = []

results, qrnn = train(depth, width, batchSize, convergence_epoch, maximum_epoch, training_data, validation_data)



In [ ]:
qrnn.save('try/qrnn_gmi_iwp.nc')

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(results['training_errors'])
ax.plot(results['validation_errors'])


### plot calibration plot

In [ ]:
qrnn =  QRNN.load('try/qrnn_gmi_iwp.nc')
x, y = qrnn.calibration(validation_data)
f, ax = plt.subplots(1, 1)
ax.plot(x, y, marker = "o", c = 'r')
ax.plot(x, x, ls = ":", c = "k")
#ax.set_xlim([0.1, 0.9])
#ax.set_ylim([0.1, 0.9])
ax.set_aspect(1.0)
ax.set_xlabel("Predicted frequency")
ax.set_ylabel("Observed frequency")

In [5]:
valdata = gmiData(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/TB_GMI_test_satdata.nc"), inChannels) 

AttributeError: NetCDF: Attribute not found

In [ ]:

y_pre = []
y = []
y_prior = []
y_pos_mean = []
x_in = []

nbatch = validation_data.__len__()
print (nbatch)
for i in range(nbatch):
    
    xx, yy = validation_data.__getitem__(i)
    
    x = xx.detach().numpy() 

    y_pre.append(qrnn.predict(x)) 
    y_pos_mean.append((qrnn.posterior_mean(x)))
       
    y.append(yy.detach().numpy())
    x_in.append(x)

In [ ]:
x_in = np.concatenate(x_in, axis = 0)
y_pre = np.concatenate(y_pre, axis = 0)
y = np.concatenate(y, axis= 0)
y_pos_mean = np.concatenate(y_pos_mean, axis = 0)

In [ ]:
plt.rcParams.update({'font.size': 20})
bins1 = np.arange(0, 30, 0.1)
fig, ax = plt.subplots(1, 1, figsize = [8, 8])
ax.hist(y_pos_mean, bins1, density = True , histtype = "step", label = "predicted")


ax.hist(y, bins1, density = True, histtype = "step", label = "actual")
ax.set_yscale('log')
ax.set_xscale('log')

ax.legend()
ax.set_ylabel("PDF")
ax.set_xlabel("IWP[kg/m2]")

In [ ]:
import scipy
from matplotlib import ticker, cm
xyrange = [[0, 15], [0, 15]] # data range

bins = [45, 45] # number of bins
hh, locx, locy = np.histogram2d(y, y_pos_mean, 
                                range=xyrange, bins=bins, density = True)
posx = np.digitize(y, locx)
posy = np.digitize(y_pos_mean, locy)

fig, ax = plt.subplots(1, 1, figsize = [10, 8])
cs = ax.contourf(np.flipud(hh.T),
                extent=np.array(xyrange).flatten(), 
            locator= ticker.LogLocator(), origin='upper')
cbar = fig.colorbar(cs)
ax.set_ylim([0, 12])
ax.set_xlim([0, 12])
xy = np.arange(0, 13, 1)
yy = xy
ax.plot(xy, yy)
ax.set_ylabel("IWP Predicted [kg/m2]")
ax.set_xlabel("IWP Observed [kg/m2]")
#ax.set_yscale('log')
#ax.set_xscale('log')